# Feature Selection with Nearest Neighbor

In [3]:
import numpy as np
import pandas as pd

In [4]:
# Read in txt file
small_data = "CS170_Small_Data__27.txt"
large_data = "CS170_Large_Data__3.txt"

def read_file(file):
    data = np.loadtxt(file)
    labels = data[:, 0] # First column
    features = data[:, 1:] 
    file = pd.DataFrame(data)
    return labels, features, file

In [5]:
# Print out dataset
labels_sm, features_sm, small_data_df = read_file(small_data)
small_data_df

,0,1,2,3,4,5,6
0,2.0,1.332495,-0.323919,1.669955,1.062248,-1.449804,-1.107199
1,2.0,-1.794856,0.526500,-0.711527,0.387848,1.896240,-0.705093
2,1.0,0.528179,0.446601,-0.751187,-1.156860,0.058414,0.085081
3,2.0,-0.841448,0.671580,-0.791409,1.650037,-0.001030,-0.926882
4,2.0,-1.336178,0.270797,0.594884,0.705356,0.280192,0.708196
...,...,...,...,...,...,...,...
495,1.0,0.393640,1.147777,0.910491,1.839495,0.745300,-1.335473
496,1.0,0.632794,-0.228989,0.661365,-1.281785,-0.243432,-0.045499
497,1.0,0.078261,-0.152425,0.304679,0.661816,1.979366,0.858024
498,2.0,0.407733,-0.590921,-1.298667,0.688415,1.611739,-0.593749


In [6]:
labels_lg, features_lg, large_data_df = read_file(large_data)
large_data_df

,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,40
0,2.0,1.337954,0.256970,-0.531852,-1.261782,0.747598,-1.948783,1.381252,-2.143330,0.153668,...,-0.859254,-0.391408,1.334797,-0.665610,1.198808,-0.201056,-1.754162,-1.601288,1.405606,-1.106220
1,2.0,-1.468259,-0.269031,-0.746182,-1.651228,0.255069,-1.344827,0.459359,-0.846903,-0.584782,...,-0.178859,2.210222,-0.663049,-1.073713,-0.964908,0.935659,0.862336,-0.448571,-0.400706,-2.310528
2,2.0,0.675852,-1.350342,0.624393,-1.050159,-0.848243,0.579179,0.439136,0.337129,-0.258679,...,-0.161698,-0.080208,1.080877,-0.145501,0.618212,-0.319206,0.123935,-0.686428,-0.165484,0.179470
3,2.0,0.952794,-0.648301,-0.895861,-0.238372,-0.372316,0.399638,1.144554,-0.330263,0.460266,...,0.604769,-0.866218,1.481476,-1.162825,0.980644,1.114730,1.118577,2.868485,-0.526972,-2.435807
4,2.0,-0.920238,-0.668906,-1.456950,-1.554868,-0.493702,-1.131406,-0.815621,-0.803982,0.838583,...,-1.275160,-1.583918,-1.036549,0.959788,-0.146866,-1.574191,0.449612,0.365045,0.267119,-0.380352
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2.0,0.062961,1.743790,-0.573885,0.578433,-1.131986,0.590080,1.168345,-0.412795,0.505087,...,1.997742,-0.551118,0.736027,-0.601704,1.043264,0.996995,0.708856,1.136065,1.590239,2.394932
996,2.0,0.221199,-0.001739,-0.181616,2.209081,-1.806687,-0.625591,0.168504,0.013600,-0.750940,...,-0.475208,0.001726,-0.318757,0.194222,0.262331,-0.532093,0.223548,1.154745,-0.698439,0.620140
997,2.0,0.738433,-1.460808,1.327582,1.127157,0.107590,1.706793,0.082267,-0.773898,0.032491,...,0.324037,-0.187021,-0.226941,0.729186,-0.131866,0.932936,-0.965954,-1.669371,-0.120499,-1.014348
998,2.0,1.394799,-0.778429,-0.798019,-0.164934,-1.343218,-0.542855,2.531499,0.368283,0.433284,...,0.068357,-2.035090,1.477234,0.187231,-0.032151,-0.167708,-0.554632,0.419898,-1.347725,-0.610219


**Pseudocode for Forward Selection**

1. Initialize selected_features = []
2. Compute default_rate and print it

3. For each feature selection step:

   a. Initialize best_feature = None, best_accuracy = 0
   
   b. Iterate over all unselected features:

      i. Temporarily add this feature

      ii. Run k-NN classifier with selected features

      iii. Compute accuracy

      iv. If accuracy is better than best_accuracy:
          - Update best_feature
          - Update best_accuracy

   c. If best_feature is found:
      - Add it to selected_features
      - Print progress
      - If accuracy is highest so far, update best_overall_accuracy and best_feature_combination

4. Print final best feature combination and accuracy


In [7]:
# Compute default rate
def compute_default_rate(labels):
    most_common_class = pd.Series(labels).mode()[0]
    return np.sum(labels == most_common_class) / len(labels)

# Compute accuracy
def compute_accuracy(predictions, labels):
    return np.mean(predictions == labels)

In [8]:
# Leave-One-Out Cross Validation for kNN
def loo_knn(train_features, train_labels, k=1):
    correct = 0
    n = len(train_labels)

    for i in range(n):
        # Use all samples except the i-th as training
        train_subset = np.delete(train_features, i, axis=0)
        label_subset = np.delete(train_labels, i, axis=0)
        
        # Use i-th sample as test
        # test_sample = train_features[i]
        test_sample = train_features[i].flatten()
        true_label = train_labels[i]

        # Compute distances to all training points
        distances = np.linalg.norm(train_subset - test_sample, axis=1)
        nearest_indices = np.argsort(distances)[:k]
        nearest_labels = label_subset[nearest_indices]
        
        # Predict the most common label among neighbors
        predicted_label = pd.Series(nearest_labels).mode()
        # If there are multiple modes, randomly choose one
        if len(predicted_label) > 1:
            predicted_label = predicted_label.sample(1).values[0]
        else:
            predicted_label = predicted_label.values[0]

        # Count correct predictions
        if predicted_label == true_label:
            correct += 1

    return correct / n  # Return accuracy

In [9]:
def forward_selection(labels, features):
    # 1. Initialize selected_features = [] (stores the current best feature subset)
    selected_features = []
    remaining_features = list(range(features.shape[1]))  # List of all feature indices
    best_overall_accuracy = 0
    best_feature_combination = []

    # 2. Compute default_rate and print it
    default_rate = compute_default_rate(labels)
    print("Default rate: ", default_rate, "\n")
    
    # 3. For each feature selection step:
    while remaining_features:
        # a. Initialize best_feature = None, best_accuracy = 0
        best_feature = None
        best_accuracy = 0
        print("Searching for the best new feature to add:")
        
        # b. Iterate over all unselected features:
        for feature in remaining_features:
            # i. Temporarily add this feature
            temp_selected_features = selected_features + [feature]
            # ii. Run k-NN classifier with selected features
            # Combine with leave-one-out cross-validation
            # iii. Compute accuracy
            accuracy = loo_knn(features[:, temp_selected_features], labels, k=1)
            print(f"Using features {temp_selected_features}, accuracy is {accuracy * 100:.1f}%")
            # iv. If accuracy is better than best_accuracy:
            if accuracy > best_accuracy:
                # - Update best_feature
                best_feature = feature
                # - Update best_accuracy
                best_accuracy = accuracy

        # c. If best_feature is found:
        if best_feature is not None:
            # - Add it to selected_features
            selected_features.append(best_feature)
            remaining_features.remove(best_feature)  # Need to remove from remaining
            # - Print progress
            print(f"Feature set {selected_features} was best, accuracy is {best_accuracy * 100:.1f}%\n")
            # - If accuracy is highest so far, update best_overall_accuracy and best_feature_combination
            if best_accuracy > best_overall_accuracy:
                best_overall_accuracy = best_accuracy
                best_feature_combination = selected_features.copy()
            else:
                print("(! Warning: Accuracy starts decreased...)\n")

    # 4. Print final best feature combination and accuracy
    print(f"Finished search! The best feature subset is {best_feature_combination}, "
        f"which has an accuracy of {best_overall_accuracy * 100:.1f}%")
        
    return best_feature_combination, best_overall_accuracy


In [10]:
# Make forward selection on small data
best_features_for_sm, best_accuracy_for_sm = forward_selection(labels_sm, features_sm)

Default rate:  0.802 

Searching for the best new feature to add:
Using features [0], accuracy is 81.6%
Using features [1], accuracy is 64.4%
Using features [2], accuracy is 72.8%
Using features [3], accuracy is 66.0%
Using features [4], accuracy is 67.6%
Using features [5], accuracy is 64.4%
Feature set [0] was best, accuracy is 81.6%

Searching for the best new feature to add:
Using features [0, 1], accuracy is 82.4%
Using features [0, 2], accuracy is 95.8%
Using features [0, 3], accuracy is 83.6%
Using features [0, 4], accuracy is 82.2%
Using features [0, 5], accuracy is 80.4%
Feature set [0, 2] was best, accuracy is 95.8%

Searching for the best new feature to add:
Using features [0, 2, 1], accuracy is 90.8%
Using features [0, 2, 3], accuracy is 89.8%
Using features [0, 2, 4], accuracy is 93.2%
Using features [0, 2, 5], accuracy is 92.4%
Feature set [0, 2, 4] was best, accuracy is 93.2%

(! Warning: Accuracy starts decreased...)

Searching for the best new feature to add:
Using fea

**Pseudocode for Backward Selection**

1. Initialize selected_features = [All Features]

2. Compute initial accuracy with all features and print it

3. For each feature elimination step:
    a. Initialize worst_feature = None, best_accuracy = 0

    b. Iterate over all selected features:


    i. Temporarily remove this feature

    ii. Run k-NN classifier with remaining features

    iii. Compute accuracy

    iv. If accuracy is better than best_accuracy:

    - Update worst_feature
    
    - Update best_accuracy

    c. If worst_feature is found:

    Remove it from selected_features

    Print progress

    If accuracy is highest so far, update best_overall_accuracy and best_feature_combination

    If accuracy decreases, stop early (optional)

4. Print final best feature combination and accuracy

In [11]:
# Compute initial accuracy
def compute_initial_accuracy(labels, features):
    return loo_knn(features, labels, k=1)

In [12]:
def backward_elimination(labels, features):
    # 1. Initialize selected_features = [All Features]
    selected_features = list(range(features.shape[1]))
    best_overall_accuracy = 0
    best_feature_combination = []

    # 2. Compute initial accuracy with all features and print it
    initial_accuracy = compute_initial_accuracy(labels, features)
    print(f"Initial accuracy: {initial_accuracy * 100:.1f}%\n")
    
    # 3. For each feature elimination step:
    while selected_features:
        # a. Initialize worst_feature = None, best_accuracy = 0
        worst_feature = None
        best_accuracy = 0
        print("Searching for the worst feature to remove:")
        
        # b. Iterate over all selected features:
        for feature in selected_features:
            # i. Temporarily remove this feature
            temp_selected_features = selected_features.copy()
            temp_selected_features.remove(feature)
            # ii. Run k-NN classifier with remaining features
            # Combine with leave-one-out cross-validation
            # iii. Compute accuracy
            accuracy = loo_knn(features[:, temp_selected_features], labels, k=1)
            print(f"Using features {temp_selected_features}, accuracy is {accuracy * 100:.1f}%")
            # iv. If accuracy is better than best_accuracy:
            if accuracy > best_accuracy:
                # - Update worst_feature
                worst_feature = feature
                # - Update best_accuracy
                best_accuracy = accuracy
        # c. If worst_feature is found:
        if worst_feature is not None:
            # Remove it from selected_features
            selected_features.remove(worst_feature)
            # Print progress
            print(f"Feature set {selected_features} was best, accuracy is {best_accuracy * 100:.1f}%\n")
            # If accuracy is highest so far, update best_overall_accuracy and best_feature_combination
            if best_accuracy > best_overall_accuracy:
                best_overall_accuracy = best_accuracy
                best_feature_combination = selected_features.copy()
            else:
                print("(! Warning: Accuracy starts decreased...)\n")

    # 4. Print final best feature combination and accuracy
    print(f"Finished search! The best feature subset is {best_feature_combination}, "
        f"which has an accuracy of {best_overall_accuracy * 100:.1f}%")
    
    return best_feature_combination, best_overall_accuracy


In [13]:
# Make backward elimination on small data
best_features_back_sm, best_accuracy_back_sm = backward_elimination(labels_sm, features_sm)

Initial accuracy: 83.2%

Searching for the worst feature to remove:
Using features [1, 2, 3, 4, 5], accuracy is 72.2%
Using features [0, 2, 3, 4, 5], accuracy is 87.6%
Using features [0, 1, 3, 4, 5], accuracy is 76.6%
Using features [0, 1, 2, 4, 5], accuracy is 85.8%
Using features [0, 1, 2, 3, 5], accuracy is 85.6%
Using features [0, 1, 2, 3, 4], accuracy is 84.4%
Feature set [0, 2, 3, 4, 5] was best, accuracy is 87.6%

Searching for the worst feature to remove:
Using features [2, 3, 4, 5], accuracy is 73.2%
Using features [0, 3, 4, 5], accuracy is 80.2%
Using features [0, 2, 4, 5], accuracy is 88.8%
Using features [0, 2, 3, 5], accuracy is 88.6%
Using features [0, 2, 3, 4], accuracy is 90.0%
Feature set [0, 2, 3, 4] was best, accuracy is 90.0%

Searching for the worst feature to remove:
Using features [2, 3, 4], accuracy is 72.4%
Using features [0, 3, 4], accuracy is 82.0%
Using features [0, 2, 4], accuracy is 93.2%
Using features [0, 2, 3], accuracy is 89.8%
Feature set [0, 2, 4] wa

Now we had tested code work for small data, let's peform on large data

In [14]:
# Make forward selection on large data
best_features_for_lg, best_accuracy_for_lg = forward_selection(labels_lg, features_lg)

Default rate:  0.807 

Searching for the best new feature to add:
Using features [0], accuracy is 68.8%
Using features [1], accuracy is 69.8%
Using features [2], accuracy is 67.8%
Using features [3], accuracy is 72.2%
Using features [4], accuracy is 71.3%
Using features [5], accuracy is 68.0%
Using features [6], accuracy is 86.4%
Using features [7], accuracy is 70.5%
Using features [8], accuracy is 66.1%
Using features [9], accuracy is 69.4%
Using features [10], accuracy is 66.6%
Using features [11], accuracy is 68.9%
Using features [12], accuracy is 68.8%
Using features [13], accuracy is 67.7%
Using features [14], accuracy is 68.7%
Using features [15], accuracy is 68.8%
Using features [16], accuracy is 67.6%
Using features [17], accuracy is 65.5%
Using features [18], accuracy is 69.0%
Using features [19], accuracy is 67.1%
Using features [20], accuracy is 71.0%
Using features [21], accuracy is 70.2%
Using features [22], accuracy is 67.2%
Using features [23], accuracy is 69.5%
Using fe

In [15]:
# Make backward elimination on large data
best_features_back_lg, best_accuracy_back_lg = backward_elimination(labels_lg, features_lg)

Initial accuracy: 65.6%

Searching for the worst feature to remove:
Using features [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39], accuracy is 65.9%
Using features [0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39], accuracy is 65.8%
Using features [0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39], accuracy is 65.6%
Using features [0, 1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39], accuracy is 67.1%
Using features [0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39], accuracy is 65.5%
Using features [0, 1, 

---

### Extra Credit

Here is the dataset I use: [Iris](https://archive.ics.uci.edu/dataset/53/iris)

   Given is the attribute name, attribute type, the measurement unit and a
   brief description.  The number of rings is the value to predict: either
   as a continuous value or as a classification problem.

	Name		Data Type	Meas.	Description
	----		---------	-----	-----------
	Sex		nominal			M, F, and I (infant)
	Length		continuous	mm	Longest shell measurement
	Diameter	continuous	mm	perpendicular to length
	Height		continuous	mm	with meat in shell
	Whole weight	continuous	grams	whole abalone
	Shucked weight	continuous	grams	weight of meat
	Viscera weight	continuous	grams	gut weight (after bleeding)
	Shell weight	continuous	grams	after being dried
	Rings		integer			+1.5 gives the age in years


In [16]:
# Read in file abalone and transfer to dataframe
abalone_data = "abalone.data"

# Use pandas to read the data, which can handle mixed data types
abalone_data_df = pd.read_csv(abalone_data, header=None, names=['Sex', 'Length', 'Diameter', 'Height', 'Whole weight', 'Shucked weight', 'Viscera weight', 'Shell weight', 'Rings'])

# Convert Sex columns to numerical values (0, 1, 2)
abalone_data_df['Sex'] = abalone_data_df['Sex'].map({'M': 0, 'F': 1, 'I': 2})

# Remove rings column
abalone_data_df = abalone_data_df.drop(columns=['Rings'])

abalone_data_df

,Sex,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight
0,0,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.1500
1,0,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.0700
2,1,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.2100
3,0,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.1550
4,2,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.0550
...,...,...,...,...,...,...,...,...
4172,1,0.565,0.450,0.165,0.8870,0.3700,0.2390,0.2490
4173,0,0.590,0.440,0.135,0.9660,0.4390,0.2145,0.2605
4174,0,0.600,0.475,0.205,1.1760,0.5255,0.2875,0.3080
4175,1,0.625,0.485,0.150,1.0945,0.5310,0.2610,0.2960


In [17]:
# Get labels and features
# Labels are the first column, features are the rest
labels_abalone = abalone_data_df.iloc[:, 0].values
features_abalone = abalone_data_df.iloc[:, 1:].values
print(labels_abalone)
print(features_abalone)

[0 0 1 ... 0 1 0]
[[0.455  0.365  0.095  ... 0.2245 0.101  0.15  ]
 [0.35   0.265  0.09   ... 0.0995 0.0485 0.07  ]
 [0.53   0.42   0.135  ... 0.2565 0.1415 0.21  ]
 ...
 [0.6    0.475  0.205  ... 0.5255 0.2875 0.308 ]
 [0.625  0.485  0.15   ... 0.531  0.261  0.296 ]
 [0.71   0.555  0.195  ... 0.9455 0.3765 0.495 ]]


In [18]:
# Normalize the data except for the first column

from sklearn.preprocessing import StandardScaler

sex_column = abalone_data_df["Sex"]
numerical_features = abalone_data_df.drop(columns=["Sex"])

# Initialize the scaler
scaler = StandardScaler()
normalized_features = pd.DataFrame(scaler.fit_transform(numerical_features), columns=numerical_features.columns)

# Recombine 'Sex' column with normalized features
normalized_df = pd.concat([sex_column, normalized_features], axis=1)
normalized_df

,Sex,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight
0,0,-0.574558,-0.432149,-1.064424,-0.641898,-0.607685,-0.726212,-0.638217
1,0,-1.448986,-1.439929,-1.183978,-1.230277,-1.170910,-1.205221,-1.212987
2,1,0.050033,0.122130,-0.107991,-0.309469,-0.463500,-0.356690,-0.207139
3,0,-0.699476,-0.432149,-0.347099,-0.637819,-0.648238,-0.607600,-0.602294
4,2,-1.615544,-1.540707,-1.423087,-1.272086,-1.215968,-1.287337,-1.320757
...,...,...,...,...,...,...,...,...
4172,1,0.341509,0.424464,0.609334,0.118813,0.047908,0.532900,0.073062
4173,0,0.549706,0.323686,-0.107991,0.279929,0.358808,0.309362,0.155685
4174,0,0.632985,0.676409,1.565767,0.708212,0.748559,0.975413,0.496955
4175,1,0.841182,0.777187,0.250672,0.541998,0.773341,0.733627,0.410739


In [19]:
# Make forward selection on abalone data
best_features_for_abalone, best_accuracy_for_abalone = forward_selection(labels_abalone, normalized_df.values[:, 1:])

Default rate:  0.3658127842949485 

Searching for the best new feature to add:


Using features [0], accuracy is 44.0%
Using features [1], accuracy is 42.6%
Using features [2], accuracy is 44.2%
Using features [3], accuracy is 43.1%
Using features [4], accuracy is 44.6%
Using features [5], accuracy is 45.5%
Using features [6], accuracy is 44.6%
Feature set [5] was best, accuracy is 45.5%

Searching for the best new feature to add:
Using features [5, 0], accuracy is 46.6%
Using features [5, 1], accuracy is 46.1%
Using features [5, 2], accuracy is 45.7%
Using features [5, 3], accuracy is 46.5%
Using features [5, 4], accuracy is 46.3%
Using features [5, 6], accuracy is 45.7%
Feature set [5, 0] was best, accuracy is 46.6%

Searching for the best new feature to add:
Using features [5, 0, 1], accuracy is 47.8%
Using features [5, 0, 2], accuracy is 46.9%
Using features [5, 0, 3], accuracy is 47.1%
Using features [5, 0, 4], accuracy is 46.8%
Using features [5, 0, 6], accuracy is 48.2%
Feature set [5, 0, 6] was best, accuracy is 48.2%

Searching for the best new feature to 

Since the accuracy it got was very low, so I used a decision tree to train the dataset again. It ends up getting similar accuracy. 

This is not a part of EC, just for testing only.

In [20]:
# Now, use sklearn, decision tree model to run this dataset, normalized_df, and get accuracy
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(normalized_df.values[:, 1:], labels_abalone, test_size=0.2, random_state=42)
# Initialize the Decision Tree Classifier
clf = DecisionTreeClassifier(random_state=42)
# Train the classifier
clf.fit(X_train, y_train)
# Make predictions on the test set
y_pred = clf.predict(X_test)
# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Decision Tree Classifier Accuracy: {accuracy * 100:.1f}%")

Decision Tree Classifier Accuracy: 49.4%
